# Notes

~ how to find trace?


# Packages

In [1]:
using ITensors

In [2]:
using Plots

In [3]:
using QuantumOptics


In [4]:
using LinearAlgebra

In [5]:
using Interpolations


In [6]:
using Metal

# Functions

## State Definition

In [22]:
function ITensors.op(::OpName"Ground" , ::SiteType"Qudit" , d::Int)
    mat = zeros(d, d)
    mat[1,1] = 1
    return (mat)
end
function ITensors.op(::OpName"Ground" , ::SiteType"Qubit" )
    mat = zeros(2, 2)
    mat[1,1] = 1
    return mat
end

In [23]:
function ITensors.op(::OpName"Excite1" , ::SiteType"Qudit" , d::Int)
    mat = zeros(d, d)
    mat[2,2] = 1
    return (mat)
end
function ITensors.op(::OpName"Excite1" , ::SiteType"Qubit")
    mat = zeros(2, 2)
    mat[2,2] = 1
    return mat
end

In [ ]:
function ITensors.op(::OpName"Sz" , ::SiteType"Qudit" , d::Int)
    mat = zeros(d, d)
    mat[1,1] = 1
    mat[2,2] = -1
    return mat
end

## Operations on the MPO (Beamsplitter/Swap/Copy)

In [ ]:
#can try beamsplitter with opsum?


In [ ]:
function beamsplitter!(MPO_i, index_1, index_2, site_list)
    #cutoff = 1E-20

    op_1 = ((op("A",sys_sites[index_1]) * op("Adag",sys_sites[index_2])) + (op("A",sys_sites[index_2]) * op("Adag",sys_sites[index_1])))
    H_ = exp((-im/4) * pi * op_1)
    H_ = MPO(H_, site_list)
    H2 = apply(H_, MPO_i ; alg="zipup")
    #easier to use apply since preserves MPO data type and doesnt change to iTensor type
    #requires iTensor be applied to an MPO in that order particularly hence double dagger
    H3 = conj(swapprime( apply( H_, swapprime(conj(H2), 1,0); alg="zipup" ), 1,0))

    MPO_i[:] = H3
    MPO_i /= tr(MPO_i)

    return nothing
end

In [ ]:
function swap!(MPO_i, index_1, index_2, site_list)
    cutoff = 1E-20

    op_1 = ((op("A",site_list[index_1]) * op("Adag",site_list[index_2])) + (op("A",site_list[index_2]) * op("Adag",site_list[index_1])))
    H_ = exp((-im/2) * pi * op_1)

    H2 = apply(H_, MPO_i ; cutoff)
    H3 = conj(swapprime( apply( H_, swapprime(conj(H2), 1,0); cutoff ), 1,0))


    MPO_i[:] = H3
    return nothing
end

In [ ]:
function n_copy_mpo(n, mpo_i, sites)
    #Assuming qudit
    l = length(mpo_i)
    new_inds = siteinds("Qudit", n*l, dim=dim(sites[1]))
    I_mpo = MPO(l*n)
    I_op = OpSum()
    for i=1:l
        for j=1:n
            ind = i + (l*(j-1))
            I_mpo[ind] = deepcopy(mpo_i[i])
            replaceind!(I_mpo[ind], sites[i], new_inds[ind])
            replaceind!(I_mpo[ind], sites[i]', new_inds[ind]')
            I_op += "I",ind
        end
    end
    #applying I to change up link indices
    I_op = MPO(I_op, new_inds)
    I_mpo = apply(I_op, I_mpo)
    I_mpo /= tr(I_mpo)
    #normalize!(I_mpo)
    return I_mpo, new_inds
end

## Calc derevative

For m output cavities
$$H(t) = H_{sys}(t) + \frac{i}{2}~\left[~~\sum_{i=1}^{m}{\left(\sqrt{\gamma}\hat{c}^\dagger + \sum_{j}^{i-1}{g_{v_j}(t)\hat{a}_j^\dagger}\right) g_{v_i}^*(t)\hat{a}_i} - h.c.~~\right]$$
We're taking system Hamiltonian as identity for the time being

In [71]:
#func calc drho___/dt
function drho(sites, rho, gv_f, rg; t=0,  deph = 0)
    #drho_(sites, rho, t)
    #(H_int * rho) + (rho * H_int) + sum(Ld rho L)

    no_cavs = length(sites) - 1

    H = OpSum()
    H += -2f0im,"I",1
    L_0 = OpSum()
    L_0 += rg,"A",1

    for i=2:no_cavs+1

        L_0 += gv_f[i-1](t),"A",i
        H += (rg*gv_f[i-1](t)),"Adag",1,"A",i
        H -= (rg*gv_f[i-1](t)),"A",1,"Adag",i

        #L_0 += exp(t/2),"A",i
        #H += (rg*exp(t/2)),"Adag",1,"A",i
        #H -= (rg*exp(t/2)),"A",1,"Adag",i

        for j=2:i-1

            H += (gv_f[j-1](t)*gv_f[i-1](t)),"Adag",j,"A",i
            H -= (gv_f[j-1](t)*gv_f[i-1](t)),"A",j,"Adag",i
        end
    end

    H = mtl(MPO(H, sites))
    #H *= im/2
    drho_ = 0.5f0*( apply(H, rho; alg="zipup") - apply(rho, H; alg="zipup") )

    #L dagger
    L_0 = mtl(MPO(L_0, sites))
    L_0d = swapprime(conj(L_0), 1,0)
    LdL = 0.5f0 * apply(L_0d , L_0; alg="zipup")

    drho_ += apply(apply(L_0 , rho; alg="zipup"), L_0d; alg="zipup")
    drho_ -= apply(LdL, rho; alg="zipup")
    drho_ -= apply(rho, LdL; alg="zipup")

    if deph!= 0 
        #println("Deph")
        L_1 = OpSum()
        L_1 += sqrt(deph),"Sz",1
        L_1 = mtl(MPO(L_1, sites))
        drho_ += apply(apply(L_1 , rho; alg="zipup"), L_1; alg="zipup")
        drho_ -= deph*rho*1f0
    end

    return drho_
end

drho (generic function with 1 method)

## Measurements/Expectations

In [83]:
function trace(MPO_i, sites_i)
    tr_ = 1f0
    for i in 1:length(sites_i)
        tr_ *= delta(sites_i[i], sites_i[i]') * MPO_i[i]
    end
    return tr_[1]
end

trace (generic function with 1 method)

In [ ]:
function ITensors.expect(MPO_i, sites_i)
    list_exp = []
    for i in 1:length(sites_i)
        append!(list_exp, trace(apply( op("n", sites_i[i]), MPO_i), sites_i))
    end
    return list_exp
end

In [ ]:
#measures <a1 a2 rho adag1 adag2> at sites 1 and 2
function measure_corel(mpo_i, sites, site1, site2)
    AA_op = OpSum()
    AA_op += "A",site1,"A",site2
    AA_op = MPO(AA_op, sites)
    AdAd_op = swapprime(conj(AA_op), 1,0)
    val = tr(apply(AA_op, apply(mpo_i, AdAd_op)))
    return val
end 

In [ ]:
#measures <a1 a2 rho adag1 adag2> at sites 1 and 2
function measure_a(mpo_i, sites, site1)
    A_op = OpSum()
    A_op += "A",site1
    A_op = MPO(A_op, sites)
    Ad_op = swapprime(conj(A_op), 1,0)
    val = tr(apply(A_op, apply(mpo_i, Ad_op)))
    return val
end 

In [ ]:
function projection(mpo_i, mps_i)

end

In [ ]:
function g_34(mpo_i, sites, no_cavs)
    # measuring G_34 t, tau integrated
    G_34 = 0.0
    denom1 = 0.0
    denom2 = 0.0
    for i=1:no_cavs
        for j=1:no_cavs
            #if v^2 integrates out to 1
            term = measure_corel(mpo_i, sites, i, no_cavs+j)
            #println(ITensors.expect(double, double_sites))
            #println(term)
            G_34 += 0.5 * term
        end
        denom1 += measure_a(mpo_i, sites, i)
        #println(ITensors.expect(mpo_i, sites))
        denom2 += measure_a(mpo_i, sites, no_cavs+i)
    end
    denom =0.5 * denom1 * denom2

    return G_34/denom
end


## Atom to cavity loop 

In [52]:
function create_MPO(no_cavs, depha, gamma, dt, t_final)

    #creating initial state and sites
    sys_sites = siteinds("Qudit", no_cavs+1, dim=3)
    input_list = repeat(["Ground",],no_cavs+1)
    input_list[1] = "Excite1"
    sys = MPO(sys_sites, input_list)

    #Generating G2
    #gamma = 1
    g_f, eval_ = g2(gamma, depha, t_final, dt/5, no_cavs)
    println("genfunctions")
    
    #expect_list = Array{Float64}(undef, Int(t_final/dt + 1), no_cavs+1)
    #expect_list *= 0
    #ldl_list = [;]
    #j = 1
    
    """
    #evolving atom-cavity system
    for i=0:dt:t_final
        d_rho , ldl = drho(sys_sites, sys, g_f, sqrt(gamma); t=i, deph=depha)
        sys +=  dt * d_rho
        sys /= tr(sys)

        push!(ldl_list, ldl)
        expect_list[j,:] = real(ITensors.expect(sys, sys_sites))
        j += 1
        #println(i)
    end
    """
    #bonddim = [;]

    for i=0:dt:t_final
        d_rho1 = drho(sys_sites, sys, g_f, sqrt(gamma); t=i, deph=depha)
        sys1 = sys + (d_rho1*dt/2)
        sys1 /=tr(sys1)
        d_rho2 = drho(sys_sites, sys1, g_f, sqrt(gamma); t=(i+(dt/2)), deph=depha)
        sys2 = sys + (d_rho2*dt/2)
        sys2 /=tr(sys2)
        d_rho3  = drho(sys_sites, sys2, g_f, sqrt(gamma); t=(i+(dt/2)), deph=depha)
        sys3 = sys + (d_rho3*dt)
        sys3 /=tr(sys3)
        d_rho4  = drho(sys_sites, sys3, g_f, sqrt(gamma); t=(i+dt), deph=depha)
        sys +=  (dt * (d_rho1 + (2f0*d_rho2) + (2f0*d_rho3) + d_rho4)/6)
        sys /= trace(sys, sys_sites)

        #push!(ldl_list, (ldl+2*ldl2+2*ldl3+ldl4)/6)
        #println("max bond dim at time: ", maximum(linkdims(sys)), " ", i)
        #append!(bonddim, maximum(linkdims(sys)))
        #expect_list[j,:] = real(ITensors.expect(sys, sys_sites))
        #j += 1
    end
    
    #trace out atom
    copy_sys = MPO(no_cavs)
    copy_sys[1] = sys[1] * delta(sys_sites[1], sys_sites[1]') * sys[2]
    for i=2:no_cavs
        copy_sys[i] = deepcopy(sys[i+1]) 
    end
    copy_sys /= tr(copy_sys)

    return copy_sys, sys_sites[2:end]
    #return sys, sys_sites
    #, g_f
    #, expect_list, ldl_list
end

create_MPO (generic function with 1 method)

## G2 function maker

In [17]:
#make a g2 function, outputs the cavity functions

function g2(gamma, deph, t_fin, dt, no_cavs)

    
    basis = SpinBasis(1//2)
    a = sigmam(basis)
    at = sigmap(basis)
    H =  identityoperator(basis)
    J = [sqrt(gamma)*a, sqrt(deph)*sigmaz(basis)]

    t_list = [0:dt:t_fin;]
    t_size = length(t_list)
    corel_m = Array{ComplexF64}(undef, t_size, t_size)
    corel_m *= 0
    
    ρ₀ = spinup(basis) ⊗ dagger(spinup(basis))
    
    tout, ρt_master = timeevolution.master(t_list, ρ₀, H, J; dt=dt)
    for i=1:t_size-1
        rhot = ρt_master[i]
        corel_m[i,i:end] = timecorrelations.correlation(t_list[1:(t_size-i+1)], rhot, H, J, at, a)
    end
    corel_m[t_size,t_size] = 0.0
    corel_m = -diagm((diag(corel_m))) + corel_m + conj(corel_m)'
    corel_m = real(corel_m)
    eigenvals, eigens = eigen(corel_m; sortby=-)
    
    gv = Dict()
    alpha = Dict()
    gv_f = [;]
    for i=1:Int(no_cavs)
        for j=1:Int(i-1)
            alpha[[i,j]] = [0.0,]
            for t=1:Int(t_size-1)
                at_ = -(gv[j][t]*eigens[t,i]*dt)
                for k=1:Int(j-1)
                    at_ -= gv[j][t]*gv[k][t]*alpha[[i,k]][t]*dt
                end
                at_ += (1-(dt*0.5*(gv[j][t])^2))*(alpha[[i,j]][end])
                push!(alpha[[i,j]], at_)

            end
        end
        v_ = eigens[:,Int(i)]

        for k=1:Int(i-1)
            v_ += (gv[k].* alpha[[i,k]])
        end
        norm = dt * cumsum(v_.^2)
        #norm[1] = 1E-3
        gv[i] = -v_./sqrt.(norm)
        #i > 1 ? gv[i][1] = 0 : nothing
        extrap = cubic_spline_interpolation(0:dt:t_fin, gv[i]; extrapolation_bc=0)
        push!(gv_f, extrap)
    end
    
    return gv_f, eigenvals[1:Int(no_cavs)]/sum(eigenvals)
end

g2 (generic function with 1 method)

# 
---

# Test create MPO with GPU

In [24]:
no_cavs = 2

2

In [92]:
sys_sites = siteinds("Qudit", no_cavs+1, dim=3)
input_list = repeat(["Ground",],no_cavs+1)
input_list[1] = "Excite1"
sys = MPO(sys_sites, input_list)
sys2 = MPO(sys_sites, input_list)

MPO
[1] ((dim=3|id=61|"Qudit,Site,n=1")', (dim=3|id=61|"Qudit,Site,n=1"), (dim=1|id=181|"Link,l=1"))
[2] ((dim=3|id=653|"Qudit,Site,n=2")', (dim=3|id=653|"Qudit,Site,n=2"), (dim=1|id=754|"Link,l=2"), (dim=1|id=181|"Link,l=1"))
[3] ((dim=3|id=836|"Qudit,Site,n=3")', (dim=3|id=836|"Qudit,Site,n=3"), (dim=1|id=754|"Link,l=2"))


In [93]:
sys_m = mtl(sys)
sys2_m = mtl(sys2)

MPO
[1] ((dim=3|id=61|"Qudit,Site,n=1")', (dim=3|id=61|"Qudit,Site,n=1"), (dim=1|id=181|"Link,l=1"))
[2] ((dim=3|id=653|"Qudit,Site,n=2")', (dim=3|id=653|"Qudit,Site,n=2"), (dim=1|id=754|"Link,l=2"), (dim=1|id=181|"Link,l=1"))
[3] ((dim=3|id=836|"Qudit,Site,n=3")', (dim=3|id=836|"Qudit,Site,n=3"), (dim=1|id=754|"Link,l=2"))


In [49]:

t = 0
H = OpSum()
H += -2im,"I",1
i = 2
j = 3


H += (g_f[i-1](t)),"Adag",1,"A",i
H -= (g_f[i-1](t)),"A",1,"Adag",i

H += (g_f[j-1](t)*g_f[i-1](t)),"Adag",j,"A",i
H -= (g_f[j-1](t)*g_f[i-1](t)),"A",j,"Adag",i


H = mtl(MPO(H, sys_sites))
#H *= im/2
drho_ = 0.5f0 *( apply(H, sys_m; alg="zipup") - apply(sys_m, H; alg="zipup") )

MPO
[1] ((dim=3|id=721|"Qudit,Site,n=1")', (dim=3|id=721|"Qudit,Site,n=1"), (dim=3|id=529|"Link,l=1"))
[2] ((dim=3|id=589|"Qudit,Site,n=2")', (dim=3|id=589|"Qudit,Site,n=2"), (dim=1|id=684|"Link,l=2"), (dim=3|id=529|"Link,l=1"))
[3] ((dim=3|id=260|"Qudit,Site,n=3")', (dim=3|id=260|"Qudit,Site,n=3"), (dim=1|id=684|"Link,l=2"))


In [51]:
drho(sys_sites, sys_m, g_f, gamma; t=0,  deph = 0)

MPO
[1] ((dim=3|id=721|"Qudit,Site,n=1")', (dim=3|id=721|"Qudit,Site,n=1"), (dim=7|id=744|"Link,l=1"))
[2] ((dim=3|id=589|"Qudit,Site,n=2")', (dim=3|id=589|"Qudit,Site,n=2"), (dim=4|id=337|"Link,l=2"), (dim=7|id=744|"Link,l=1"))
[3] ((dim=3|id=260|"Qudit,Site,n=3")', (dim=3|id=260|"Qudit,Site,n=3"), (dim=4|id=337|"Link,l=2"))


In [56]:
dt = 0.1f0
t_final = 10f0

10.0f0

In [66]:
d_rho = drho(sys_sites, sys_m , g_f, sqrt(gamma); t=0)

MPO
[1] ((dim=3|id=721|"Qudit,Site,n=1")', (dim=3|id=721|"Qudit,Site,n=1"), (dim=15|id=693|"Link,l=1"))
[2] ((dim=3|id=589|"Qudit,Site,n=2")', (dim=3|id=589|"Qudit,Site,n=2"), (dim=8|id=70|"Link,l=2"), (dim=15|id=693|"Link,l=1"))
[3] ((dim=3|id=260|"Qudit,Site,n=3")', (dim=3|id=260|"Qudit,Site,n=3"), (dim=8|id=70|"Link,l=2"))


In [99]:
apply(sys_m , sys)

ErrorException: Setting the type parameter of the type `DenseVector` at position `NDTensors.SetParameters.Position{1}()` to `Float64` is not currently defined. Either that type parameter position doesn't exist in the type, or `set_parameter` has not been overloaded for this type.

In [101]:
apply(deepcopy(sys_m) , sys)

ErrorException: Setting the type parameter of the type `DenseVector` at position `NDTensors.SetParameters.Position{1}()` to `Float64` is not currently defined. Either that type parameter position doesn't exist in the type, or `set_parameter` has not been overloaded for this type.

In [94]:
for i=1:length(sys)
    copyto!(sys_m[i], sys[i])
end

ErrorException: Setting the type parameter of the type `DenseVector` at position `NDTensors.SetParameters.Position{1}()` to `Float64` is not currently defined. Either that type parameter position doesn't exist in the type, or `set_parameter` has not been overloaded for this type.

In [84]:
trace(sys_m, sys_sites)

ErrorException: Metal does not support Float64 values, try using Float32 instead

In [79]:

#evolving atom-cavity system
for i=0:dt:t_final
    d_rho = drho(sys_sites, sys_m , g_f, sqrt(gamma); t=0)
    sys_m +=  dt * d_rho
    sys_m = sys_m/trace(sys_m)
end


ErrorException: Metal does not support Float64 values, try using Float32 instead

In [ ]:

for i=0:dt:t_final
    d_rho1 = drho(sys_sites, sys_m, g_f, sqrt(gamma); t=i, deph=depha)
    sys1 = sys + (d_rho1*dt/2)
    sys1 /=tr(sys1)
    d_rho2 = drho(sys_sites, sys1, g_f, sqrt(gamma); t=(i+(dt/2)), deph=depha)
    sys2 = sys + (d_rho2*dt/2)
    sys2 /=tr(sys2)
    d_rho3  = drho(sys_sites, sys2, g_f, sqrt(gamma); t=(i+(dt/2)), deph=depha)
    sys3 = sys + (d_rho3*dt)
    sys3 /=tr(sys3)
    d_rho4  = drho(sys_sites, sys3, g_f, sqrt(gamma); t=(i+dt), deph=depha)
    sys_m +=  (dt * (d_rho1 + (2f0*d_rho2) + (2f0*d_rho3) + d_rho4)/6)
    sys_m /= trace(sys_m, sys_sites)
end

0.1f0

In [61]:
x = 9

9

In [64]:
gamma=1f0
depha=0f0
t_final=10f0
dt = 1f-1 

#g_f, eval_ = g2(gamma, depha, t_final, dt/5, no_cavs)


0.1f0